In [1]:
from functools import partial
import numpy as np
from scipy.optimize import root_scalar, minimize, newton, root_scalar, brentq, basinhopping
import nidaqmx as ni
from K10CR1 import K10CR1_stage
import matplotlib.pyplot as plt
import pyvisa as pv
from tqdm import tqdm
import time

def read(DAQ_name='Dev2', verbose=False, nsamples=1000):

    with ni.Task() as read_task:
        
        read_task.ai_channels.add_ai_voltage_chan(DAQ_name + "/ai4",
                                    min_val=-10, max_val=10)
        result = np.mean(np.array(read_task.read(nsamples)))
    if verbose:
        print(result)
    return result

In [2]:
rm = pv.ResourceManager()
Laser = rm.open_resource('USB0::0x1313::0x804F::M00332686::INSTR')
Laser.write(f"source1:current:level:amplitude 0.08")


hwp = K10CR1_stage(serial_no=55422054, verbose=False)
hwp.initialize_instrument()
hwp.home()
qwp = K10CR1_stage(serial_no=55425654, verbose=False)
qwp.initialize_instrument()
qwp.home()
zero_hwp = K10CR1_stage(serial_no=55425494, verbose=False)
zero_hwp.initialize_instrument()
zero_hwp.home()

Important: make sure you are not running Kinesis software in the meantime. 
Otherwise the initialization will fail.
Important: make sure you are not running Kinesis software in the meantime. 
Otherwise the initialization will fail.
Important: make sure you are not running Kinesis software in the meantime. 
Otherwise the initialization will fail.


In [3]:
def rotate(optic,
           Laser=Laser,
           start=0,
           end=180, 
           num=31,
           plot=True):
    
    Laser.write('output:state 1')
    angles = np.linspace(start,end,num)
    readings = np.zeros(shape=(num))
    
    # Stablize
    time.sleep(1)

    for index, angle in enumerate(angles):
        optic.move(angle)
        reading = read()
        readings[index] = reading
    
    Laser.write('output:state 0')

    if plot: 
        fig, ax = plt.subplots(subplot_kw={'projection':'polar'})
        ax.plot(angles / 180.0 * np.pi, readings)
        ax.set_ylim(0,)
        plt.show()

    return angles * np.pi / 180, readings

In [4]:
incident_angles, qwp_angles, hwp_angles = np.load('sorted_pol_qwp_hwp.npy').T

In [ ]:
zero_hwp_angles = []
zero_hwp_readings = []
for index, incident_angle in enumerate(incident_angles):
    print(index, incident_angle)

    # incident_angle = incident_angles[index]
    qwp_angle = qwp_angles[index]
    hwp_angle = hwp_angles[index]

    qwp.move(qwp_angle)
    hwp.move(hwp_angle)

    test_angles, test_readings = rotate(optic=zero_hwp, start=0, end=180, num=61)
    initial_zero_angle = test_angles[np.argmin(np.abs(test_readings))] * 180.0 / np.pi

    zero_hwp_readings.append([test_readings])

    test_angles, test_readings = rotate(optic=zero_hwp, start=initial_zero_angle-15, end=initial_zero_angle+15, num=31)
    zero_angle = test_angles[np.argmin(np.abs(test_readings))] * 180.0 / np.pi

    zero_hwp_angles.append([zero_angle, incident_angle, qwp_angle, hwp_angle])

    qwp.home()
    hwp.home()
    zero_hwp.home()

zero_hwp_angles
zero_hwp_angles = np.array(zero_hwp_angles)

np.save('mos2_calibration_zerohwp_pol_qwp_hwp', zero_hwp_angles)